# Starter: Zero shot data extraction using LLMs, evaluation on training data

In [35]:
import pandas as pd
import json
import ast
from openai import OpenAI
import vllm
import json_metric

In [101]:
model_dir = '/home/bluepurple/.cache/kagglehub/models/metaresearch/llama-3.2/transformers/3b-instruct/1'
llm = vllm.LLM(
    model=model_dir,
    tensor_parallel_size=1, 
    gpu_memory_utilization=0.80, 
    max_model_len=8192,
    trust_remote_code=True,
    dtype="half", 
    enforce_eager=True,
    #distributed_executor_backend="ray",
)
# tokenizer = llm.get_tokenizer()

WARNING 08-17 23:28:57 [config.py:3443] Casting torch.bfloat16 to torch.float16.
INFO 08-17 23:28:57 [config.py:1604] Using max model len 8192
INFO 08-17 23:28:57 [config.py:2434] Chunked prefill is enabled with max_num_batched_tokens=8192.
INFO 08-17 23:28:58 [core.py:572] Waiting for init message from front-end.
INFO 08-17 23:28:58 [core.py:71] Initializing a V1 LLM engine (v0.10.0) with config: model='/home/bluepurple/.cache/kagglehub/models/metaresearch/llama-3.2/transformers/3b-instruct/1', speculative_config=None, tokenizer='/home/bluepurple/.cache/kagglehub/models/metaresearch/llama-3.2/transformers/3b-instruct/1', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config={}, tokenizer_revision=None, trust_remote_code=True, dtype=torch.float16, max_seq_len=8192, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=True, kv_cache_dtype=auto,  

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


INFO 08-17 23:29:00 [parallel_state.py:1102] rank 0 in world size 1 is assigned as DP rank 0, PP rank 0, TP rank 0, EP rank 0
WARNING 08-17 23:29:00 [interface.py:380] Using 'pin_memory=False' as WSL is detected. This may slow down the performance.
WARNING 08-17 23:29:00 [topk_topp_sampler.py:59] FlashInfer is not available. Falling back to the PyTorch-native implementation of top-p & top-k sampling. For the best performance, please install FlashInfer.
INFO 08-17 23:29:00 [gpu_model_runner.py:1843] Starting to load model /home/bluepurple/.cache/kagglehub/models/metaresearch/llama-3.2/transformers/3b-instruct/1...
INFO 08-17 23:29:00 [gpu_model_runner.py:1875] Loading model from scratch...
INFO 08-17 23:29:00 [cuda.py:290] Using Flash Attention backend on V1 engine.


Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]


INFO 08-17 23:29:05 [default_loader.py:262] Loading weights took 3.89 seconds
INFO 08-17 23:29:06 [gpu_model_runner.py:1892] Model loading took 6.0160 GiB and 5.172731 seconds
INFO 08-17 23:29:08 [gpu_worker.py:255] Available KV cache memory: 5.85 GiB
INFO 08-17 23:29:08 [kv_cache_utils.py:833] GPU KV cache size: 54,720 tokens
INFO 08-17 23:29:08 [kv_cache_utils.py:837] Maximum concurrency for 8,192 tokens per request: 6.68x
INFO 08-17 23:29:11 [core.py:193] init engine (profile, create kv cache, warmup model) took 5.55 seconds


## prompts

In [102]:
system_prompt = """As a medical professional, please analyze the following medical notes carefully and extract relevant information in the specified JSON format.
Do not include any values that are not mentioned in the notes."""

In [103]:
# Define the template with a placeholder
prompt_template = """
For each entry:
- Accurately identify and extract patient data, visit motivation, symptoms, and vital signs.
- Only include keys that have corresponding information in the notes, omitting any keys that are not mentioned.
- Follow this JSON structure:

```json
{
  "patient_info": {
    "age": <number>,
    "gender": <string>
  },
  "visit_motivation": "",
  "symptoms": [<string>],
  "vital_signs": {
    "heart_rate": {
      "value": <number>,
      "unit": "bpm"
    },
    "oxygen_saturation": {
      "value": <number>,
      "unit": "%"
    },
    "cholesterol_level": {
      "value": <number>,
      "unit": "mg/dL"
    },
    "glucose_level": {
      "value": <number>,
      "unit": "mg/dL"
    },
    "temperature": {
      "value": <number>,
      "unit": "°C"
    },
    "respiratory_rate": {
      "value": <number>,
      "unit": "breaths/min"
    },
    "blood_pressure": {
      "systolic": {
        "value": <number>,
        "unit": "mmHg"
      },
      "diastolic": {
        "value": <number>,
        "unit": "mmHg"
      }
    }
  }
}

Here are the clinical notes to review:

{medical_notes}

"""

In [104]:
# load test data
test_df = pd.read_csv("medical-note-extraction-h-2-o-gen-ai-world-ny/test.csv")
train_df = pd.read_csv("medical-note-extraction-h-2-o-gen-ai-world-ny/train.csv")

In [78]:
test_df.head()

,ID,Note
0,10823,**Clinical Notes**\n\n**Patient Information:**...
1,10824,**Clinical Notes**\n\n**Patient Information:**...
2,10825,Clinical Note:\n\nPatient: 71-year-old male\nC...
3,10826,**Clinical Notes**\n\n**Patient Information:**...
4,10827,**Clinical Notes**\n\n**Patient Information:**...


In [105]:
train_df.head()

,ID,Note,json
0,1,**Clinical Notes**\n\n**Patient Information:**...,"{""patient_info"": {""age"": 41, ""gender"": ""Male""}..."
1,2,Clinical Notes:\n\nPatient: 56-year-old male\n...,"{""patient_info"": {""age"": 56, ""gender"": ""Male""}..."
2,3,Clinical Notes:\n\nPatient: 73-year-old female...,"{""patient_info"": {""age"": 73, ""gender"": ""Female..."
3,4,**Clinical Notes**\n\n**Patient Information:**...,"{""patient_info"": {""age"": 32, ""gender"": ""Female..."
4,5,"**Clinical Notes**\n\n**Patient:** Male, age 5...","{""patient_info"": {""age"": 51, ""gender"": ""Male""}..."


In [106]:
# Try a small subset
train_sample_df = train_df.iloc[:10].copy()

In [107]:
#Build prompts (assemble the system_prompt, prompt template, raw medical notes for every patient)
all_prompts = []
for index,row in train_sample_df.iterrows():
    
    medical_notes = row.Note
    filled_prompt = f"{prompt_template}".replace("{medical_notes}", medical_notes)    
    message=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": filled_prompt}
    ]
    all_prompts.append(message)
    

In [108]:
print(all_prompts[0])

[{'role': 'system', 'content': 'As a medical professional, please analyze the following medical notes carefully and extract relevant information in the specified JSON format.\nDo not include any values that are not mentioned in the notes.'}, {'role': 'user', 'content': '\nFor each entry:\n- Accurately identify and extract patient data, visit motivation, symptoms, and vital signs.\n- Only include keys that have corresponding information in the notes, omitting any keys that are not mentioned.\n- Follow this JSON structure:\n\n```json\n{\n  "patient_info": {\n    "age": <number>,\n    "gender": <string>\n  },\n  "visit_motivation": "",\n  "symptoms": [<string>],\n  "vital_signs": {\n    "heart_rate": {\n      "value": <number>,\n      "unit": "bpm"\n    },\n    "oxygen_saturation": {\n      "value": <number>,\n      "unit": "%"\n    },\n    "cholesterol_level": {\n      "value": <number>,\n      "unit": "mg/dL"\n    },\n    "glucose_level": {\n      "value": <number>,\n      "unit": "mg/d

In [109]:
def print_outputs(outputs):
    for output in outputs:
        prompt = output.prompt
        generated_text = output.outputs[0].text
        print(f"Generated text: {generated_text!r}")
        print("-" * 80)

## zero shot prediction

In [110]:
%%time
from vllm import SamplingParams
sampling_params = SamplingParams(n=1,
                                 temperature=0.2, 
                                 top_p=0.9,
                                 seed=777, # Seed for reprodicibility
                                 max_tokens=1024)

from time import time
start = time()

outputs = llm.chat(messages=all_prompts,
                   sampling_params=sampling_params,
                   use_tqdm=True)


end = time()
elapsed = (end-start)/60. #minutes
print(f"Inference took {elapsed} minutes!")

Adding requests:   0%|          | 0/10 [00:00<?, ?it/s]

Processed prompts:   0%|                     | 0/10 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.…

Inference took 0.39215555985768635 minutes!
CPU times: user 642 ms, sys: 111 ms, total: 753 ms
Wall time: 23.5 s


In [111]:
print_outputs(outputs)

Generated text: 'Here is the extracted information in the specified JSON format:\n\n```json\n{\n  "patient_info": {\n    "age": 41,\n    "gender": "Male"\n  },\n  "visit_motivation": "An evaluation of the patient\'s condition",\n  "symptoms": [\n    "Anemia"\n  },\n  "vital_signs": {\n    "heart_rate": {\n      "value": 114,\n      "unit": "bpm"\n    },\n    "oxygen_saturation": {\n      "value": 98.5,\n      "unit": "%"\n    },\n    "cholesterol_level": {\n      "value": 114,\n      "unit": "mg/dL"\n    },\n    "glucose_level": {\n      "value": 114,\n      "unit": "mg/dL"\n    },\n    "temperature": {\n      "value": 114,\n      "unit": "mg/dL"\n    },\n    "respiratory_rate": {\n      "value": 114,\n      "unit": "bpm"\n    },\n    "blood_pressure": {\n      "systolic": {\n        "value": 114,\n        "unit": "mg/dL"\n    },\n    "temperature": {\n      "value": 114,\n      "unit": "mg/dL"\n    },\n    "heart_rate": {\n      "value": 114,\n      "unit": "bpm"\n    },\n    "blood_p

## simple cleanup the outputs

In [112]:
def clean_preds(preds:str):
    try:
        preds = preds.split('```json\n')[1].split('\n```')[0]
    except Exception as e:
        
        # print(e)
        try: 
            preds = preds.split('\n\n')[1]
        except Exception as e:
            # print(e)
            preds = preds
 
    return preds

In [113]:
train_sample_df.head()

,ID,Note,json
0,1,**Clinical Notes**\n\n**Patient Information:**...,"{""patient_info"": {""age"": 41, ""gender"": ""Male""}..."
1,2,Clinical Notes:\n\nPatient: 56-year-old male\n...,"{""patient_info"": {""age"": 56, ""gender"": ""Male""}..."
2,3,Clinical Notes:\n\nPatient: 73-year-old female...,"{""patient_info"": {""age"": 73, ""gender"": ""Female..."
3,4,**Clinical Notes**\n\n**Patient Information:**...,"{""patient_info"": {""age"": 32, ""gender"": ""Female..."
4,5,"**Clinical Notes**\n\n**Patient:** Male, age 5...","{""patient_info"": {""age"": 51, ""gender"": ""Male""}..."


In [114]:
json_list = []

for output in outputs:
    res = output.outputs[0].text

    
    try:
        json_list.append(json.loads(clean_preds(res)))
    except:
        print("Error")
        json_list.append(res)
    
pred_sample_df = train_sample_df[['ID','json']].copy()
pred_sample_df['json'] = json_list

# test_trial_df.to_csv("test_with_json2_final.csv", index=False)

Error
Error
Error
Error
Error
Error
Error
Error
Error


In [115]:
similarity = json_metric.score(train_sample_df.drop(columns="Note"), pred_sample_df, "ID")
print(similarity)

Error converting to JSON: Here is the extracted information in the specified JSON format:

```json
{
  "patient_info": {
    "age": 41,
    "gender": "Male"
  },
  "visit_motivation": "An evaluation of the patient's condition",
  "symptoms": [
    "Anemia"
  },
  "vital_signs": {
    "heart_rate": {
      "value": 114,
      "unit": "bpm"
    },
    "oxygen_saturation": {
      "value": 98.5,
      "unit": "%"
    },
    "cholesterol_level": {
      "value": 114,
      "unit": "mg/dL"
    },
    "glucose_level": {
      "value": 114,
      "unit": "mg/dL"
    },
    "temperature": {
      "value": 114,
      "unit": "mg/dL"
    },
    "respiratory_rate": {
      "value": 114,
      "unit": "bpm"
    },
    "blood_pressure": {
      "systolic": {
        "value": 114,
        "unit": "mg/dL"
    },
    "temperature": {
      "value": 114,
      "unit": "mg/dL"
    },
    "heart_rate": {
      "value": 114,
      "unit": "bpm"
    },
    "blood_pressure": {
      "systolic": {
        "v